<a href="https://colab.research.google.com/github/yongchanzzz/AF3ANALYSIS/blob/main/AF3ANALYSIS_csv2cxc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AF3ANALYSIS CSV to ChimeraX commands
This notebook processes CSV files exported from the AF3ANALYSIS website to generate ChimeraX distance commands.


In [ ]:
#@title Import necessary libraries and upload files
import re
import csv
import pandas as pd
from google.colab import files
from datetime import datetime

uploaded_files = files.upload()

In [ ]:
#@title PAE cutoff
pae_cutoff = 5.0 #@param

In [ ]:
#@title Run codes and generate csv

# Step 2: Get the file name
file_name = list(uploaded_files.keys())[0]

# Step 3: Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_name)

# Step 4: Sort the data by 'min_inter_res_pae'
data_sorted = data.sort_values(by='min_inter_res_pae')

# Step 5: Filter rows where 'min_inter_res_pae' <= 5.0
data_filtered = data_sorted[data_sorted['min_inter_res_pae'] <= pae_cutoff].copy()

# Step 6: Standardize pairs to remove duplicates
data_filtered.loc[:, 'pair'] = data_filtered.apply(
    lambda row: tuple(sorted([(row['chain_1'], row['res_1_ix']), (row['chain_2'], row['res_2_ix'])])),
    axis=1
)
data_unique = data_filtered.drop_duplicates(subset=['pair']).copy()

# Step 7: Select the required columns
selected_columns = data_unique[['chain_1', 'chain_2', 'res_1_ix', 'res_2_ix']].copy()

# Step 8: Format the data for ChimeraX commands
def format_chimerax_command(row):
    return f"distance\t/{row['chain_1']}:{row['res_1_ix']}@CA\t/{row['chain_2']}:{row['res_2_ix']}@CA"

selected_columns['chimerax_command'] = selected_columns.apply(format_chimerax_command, axis=1)

In [ ]:
#@title Automatic download

# Save the formatted commands to a text file
now = datetime.now()
timestamp_str = now.strftime('%Y%m%d_%H%M%S')
output_file_path = f"AF3ANALYSIS_{timestamp_str}_PAE{pae_cutoff}.cxc"
selected_columns['chimerax_command'].to_csv(output_file_path, index=False, header=False)

print(f"ChimeraX commands saved to: {output_file_path}")
files.download(output_file_path)